# Case Review

In [5]:
import os, sys
import pandas as pd

sys.path.append('../src')
import config
import utils

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
df_results = pd.read_csv('../data/llm_results.csv')

In [7]:
df_raw = utils.get_raw_dataset()
df = utils.filter_data(df_raw)
print('dataset size: ', df.shape)
df.sample(3)

dataset size:  (24, 14)


,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color
1163633,1163633,kodak photo paper 8.5 x 11 glossy,58953,B01M0L2WLF,us,E,0,1,train,"Photo Paper, 6.5 mil, Glossy, 8-1/2 x 11, 100 ...",Perfect for everyday photo printing. Instant d...,Sold as 100 Sheets/Pack.\nInstant dry.\nNo sme...,KODAK,None
660833,660833,"dewalt 8v max cordless screwdriver kit, gyrosc...",32814,B07S7F53YK,us,E,1,1,train,DEWALT DCF680N2 8V Max Gyroscopic Screwdriver ...,None,Motion-activated Variable Speed & Reversing Co...,DEWALT,None
1163629,1163629,kodak photo paper 8.5 x 11 glossy,58953,B085F42SV6,us,E,0,1,train,"Kodak photo paper 8.5 x 11 matte, 100 count 39...",None,None,KODAK,None


In [8]:
llm = utils.get_llm()

llama_model_load_from_file_impl: using device Metal (Apple M1) - 5455 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/mckenziequinn/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q3_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader:

print_info: n_head_kv        = 8
print_info: n_rot            = 128
print_info: n_swa            = 0
print_info: n_embd_head_k    = 128
print_info: n_embd_head_v    = 128
print_info: n_gqa            = 4
print_info: n_embd_k_gqa     = 1024
print_info: n_embd_v_gqa     = 1024
print_info: f_norm_eps       = 0.0e+00
print_info: f_norm_rms_eps   = 1.0e-05
print_info: f_clamp_kqv      = 0.0e+00
print_info: f_max_alibi_bias = 0.0e+00
print_info: f_logit_scale    = 0.0e+00
print_info: n_ff             = 14336
print_info: n_expert         = 0
print_info: n_expert_used    = 0
print_info: causal attn      = 1
print_info: pooling type     = 0
print_info: rope type        = 0
print_info: rope scaling     = linear
print_info: freq_base_train  = 1000000.0
print_info: freq_scale_train = 1
print_info: n_ctx_orig_yarn  = 32768
print_info: rope_finetuned   = unknown
print_info: ssm_d_conv       = 0
print_info: ssm_d_inner      = 0
print_info: ssm_d_state      = 0
print_info: ssm_dt_rank      = 0
print_i

In [10]:
working_scenarios = ['B01G1RYHAO', 'B00EUHAGX0', 'B000EZTYHG']
results_working = []
df_working = df[df.product_id.isin(working_scenarios)]
for idx, row in df_working.iterrows():
    product_attributes = utils.json_parse(utils.extract_product_attributes(llm, row['product_title']))
    query_attributes = utils.json_parse(utils.extract_query_attributes(llm, row['query'], list(product_attributes.keys())))

    response= utils.json_parse(utils.compare_jsons(llm, query_attributes, product_attributes))
    if response['satisfied'] == True:
            corrected_query = response['fixed_query']
            results_working.append([row['query_id'], row['product_id'], response['satisfied'], row['query']])
    else:
        corrected_query = utils.get_corrected_query(llm, response['fixed_query'])
        results_working.append([row['query_id'], row['product_id'], response['satisfied'], corrected_query])
    
    print(f'query_dict: {query_attributes}')
    print(f'product_dict: {product_attributes}')
    print(f'is_correct: {response["satisfied"]}')
    print(f'corrected_query: {corrected_query}')

    print('Previous Result:')
    print(df_results[(df_results.query_id == row.query_id) & (df_results.product_id == row.product_id)])
    print(f'{len(results_working)} of {len(df_working)}')
    print ('-' * 30)

llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =    9572.08 ms /    63 tokens (  151.94 ms per token,     6.58 tokens per second)
llama_perf_context_print:        eval time =  196303.32 ms /    38 runs   ( 5165.88 ms per token,     0.19 tokens per second)
llama_perf_context_print:       total time =  205932.84 ms /   101 tokens
Llama.generate: 7 prefix-match hit, remaining 382 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =   15557.29 ms /   382 tokens (   40.73 ms per token,    24.55 tokens per second)
llama_perf_context_print:        eval time =  137581.85 ms /    26 runs   ( 5291.61 ms per token,     0.19 tokens per second)
llama_perf_context_print:       total time =  153175.34 ms /   408 tokens
Llama.generate: 3 prefix-match hit, remaining 345 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: p

query_dict: {'Battery_Type': 'aa', 'Count': '100'}
product_dict: {'Brand': 'Energizer', 'Battery_Type': 'Alkaline', 'Count': '100'}
is_correct: True
corrected_query: {'Battery_Type': 'aa', 'Count': '100'}
Previous Result:
   query_id  product_id  is_correct        corrected_query
0      6014  B01G1RYHAO        True  aa batteries 100 pack
1 of 3
------------------------------


llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =    8731.97 ms /    59 tokens (  148.00 ms per token,     6.76 tokens per second)
llama_perf_context_print:        eval time =  265637.54 ms /    51 runs   ( 5208.58 ms per token,     0.19 tokens per second)
llama_perf_context_print:       total time =  274441.03 ms /   110 tokens
Llama.generate: 7 prefix-match hit, remaining 396 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =   16035.26 ms /   396 tokens (   40.49 ms per token,    24.70 tokens per second)
llama_perf_context_print:        eval time =  191222.57 ms /    37 runs   ( 5168.18 ms per token,     0.19 tokens per second)
llama_perf_context_print:       total time =  207308.34 ms /   433 tokens
Llama.generate: 3 prefix-match hit, remaining 365 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: p

query_dict: {'Brand': 'dewalt', 'Type': 'cordless screwdriver', 'Power Source': '8v max'}
product_dict: {'Brand': 'DEWALT', 'Type': 'Battery Charger', 'Power Source': '8V MAX', 'Battery Voltage': '8V'}
is_correct: False
corrected_query: 8V MAX DEWALT Battery Charger
Previous Result:
    query_id  product_id  is_correct                corrected_query
11     32814  B00EUHAGX0       False  8V MAX DEWALT Battery Charger
2 of 3
------------------------------


llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =   12209.40 ms /    85 tokens (  143.64 ms per token,     6.96 tokens per second)
llama_perf_context_print:        eval time =  368809.28 ms /    70 runs   ( 5268.70 ms per token,     0.19 tokens per second)
llama_perf_context_print:       total time =  381116.15 ms /   155 tokens
Llama.generate: 7 prefix-match hit, remaining 399 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =   16781.06 ms /   399 tokens (   42.06 ms per token,    23.78 tokens per second)
llama_perf_context_print:        eval time =  214035.07 ms /    40 runs   ( 5350.88 ms per token,     0.19 tokens per second)
llama_perf_context_print:       total time =  230877.29 ms /   439 tokens
Llama.generate: 3 prefix-match hit, remaining 379 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: p

query_dict: {'Product_Type': 'Photo Paper', 'Size': {'Width': 8.5, 'Height': 11}}
product_dict: {'Brand': 'Kodak', 'Product_Type': 'Photo Paper', 'Finish': 'Gloss', 'Thickness': '7 mil', 'Size': {'Width': 8.5, 'Length': 11}}
is_correct: True
corrected_query: {'Product_Type': 'Photo Paper'}
Previous Result:
    query_id  product_id  is_correct                    corrected_query
18     58953  B000EZTYHG        True  kodak photo paper 8.5 x 11 glossy
3 of 3
------------------------------


In [11]:
error_scenarios = ['B085F42SV6','B00DL7QDS2']
results_error = []
df_error = df[df.product_id.isin(error_scenarios)]
for idx, row in df_error.iterrows():
    product_attributes = utils.json_parse(utils.extract_product_attributes(llm, row['product_title']))
    query_attributes = utils.json_parse(utils.extract_query_attributes(llm, row['query'], list(product_attributes.keys())))

    response= utils.json_parse(utils.compare_jsons(llm, query_attributes, product_attributes))
    if response['satisfied'] == True:
            corrected_query = response['fixed_query']
            results_error.append([row['query_id'], row['product_id'], response['satisfied'], row['query']])
    else:
        corrected_query = utils.get_corrected_query(llm, response['fixed_query'])
        results_error.append([row['query_id'], row['product_id'], response['satisfied'], corrected_query])
    
    print(f'query_dict: {query_attributes}')
    print(f'product_dict: {product_attributes}')
    print(f'is_correct: {response["satisfied"]}')
    print(f'corrected_query: {corrected_query}')

    print('Previous Result:')
    print(df_results[(df_results.query_id == row.query_id) & (df_results.product_id == row.product_id)])
    print(f'{len(results_error)} of {len(df_error)}')
    print ('-' * 30)

Llama.generate: 3 prefix-match hit, remaining 76 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =    9998.10 ms /    76 tokens (  131.55 ms per token,     7.60 tokens per second)
llama_perf_context_print:        eval time =  382051.79 ms /    71 runs   ( 5381.01 ms per token,     0.19 tokens per second)
llama_perf_context_print:       total time =  392152.49 ms /   147 tokens
Llama.generate: 7 prefix-match hit, remaining 403 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =   18273.97 ms /   403 tokens (   45.34 ms per token,    22.05 tokens per second)
llama_perf_context_print:        eval time =  190808.06 ms /    37 runs   ( 5156.97 ms per token,     0.19 tokens per second)
llama_perf_context_print:       total time =  209137.83 ms /   440 tokens
Llama.generate: 3 prefix-match hit, remaining 382 prompt tokens to eval
llama_perf_cont

query_dict: {'Brand': 'dewalt', 'Type': 'cordless screwdriver', 'Power Source': '8v max'}
product_dict: {'Brand': 'DEWALT', 'Type': 'Cordless Screwdriver Kit', 'Power Source': '8V MAX', 'Features': ['Gyroscopic'], 'Batteries Included': True, 'Quantity': 2}
is_correct: False
corrected_query: 8V MAX DEWALT Cordless Screwdriver Kit.
Previous Result:
    query_id  product_id  is_correct                          corrected_query
12     32814  B00DL7QDS2       False  8V MAX DEWALT Cordless Screwdriver Kit.
1 of 2
------------------------------


llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =   10028.77 ms /    87 tokens (  115.27 ms per token,     8.68 tokens per second)
llama_perf_context_print:        eval time =  205720.81 ms /    38 runs   ( 5413.71 ms per token,     0.18 tokens per second)
llama_perf_context_print:       total time =  215791.76 ms /   125 tokens
Llama.generate: 7 prefix-match hit, remaining 385 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: prompt eval time =   17667.07 ms /   385 tokens (   45.89 ms per token,    21.79 tokens per second)
llama_perf_context_print:        eval time =  216968.22 ms /    40 runs   ( 5424.21 ms per token,     0.18 tokens per second)
llama_perf_context_print:       total time =  234685.11 ms /   425 tokens
Llama.generate: 3 prefix-match hit, remaining 353 prompt tokens to eval
llama_perf_context_print:        load time =    9577.53 ms
llama_perf_context_print: p

query_dict: {'Brand': 'Kodak', 'Size': {'Width': 8.5, 'Height': 11}}
product_dict: {'Brand': 'Kodak', 'Size': {'Width': 8.5, 'Length': 11}}
is_correct: True
corrected_query: {'Brand': 'Kodak', 'Size': {'Width': 8.5, 'Height': 11}}
Previous Result:
    query_id  product_id  is_correct                    corrected_query
15     58953  B085F42SV6        True  kodak photo paper 8.5 x 11 glossy
2 of 2
------------------------------
